# Simulation

Each simulation is encapsulated as an object that is responsible for
finding the location of each dataset in the simulation, loading it —
which may involve downloading and caching the relevant file — and
returning an object that can be used to access that piece of data.


## Location

We begin by simply loading the simulation.  There are three pieces of
information needed to identify a simulation:

  1. SXS ID

     This identifies the simulation type and includes a number.  For
     example, "SXS:BBH:0123", which identifies this simulation as a
     product of the SXS collaboration, specifies that it is a binary
     black hole simulation, and that it has been assigned the number
     "0123" in that series.  Note that these numbers are not
     necessarily sequential.  Nor do they necessarily correspond to
     the relative age of the simulation; that information is contained
     in the metadata itself in the various `date_*` keys.

  2. Version
     
     This identifies the version of the data — like "v2.0".  Unlike
     the SXS ID, this is an optional specifier.  If not provided, the
     most recent version is used.  All versions refer to the same
     underlying simulation, but the raw data may have been processed
     differently, may be provided in incompatible formats, etc.  For
     exploratory work, it is often convenient to simply use the most
     recent version.  However, for reproducibility, it is important to
     specify the version of the data you use for a given analysis.

   3. Lev (resolution)
      
      This identifies the resolution of the simulation — like "Lev5".
      This is also optional.  If not provided, the highest resolution
      is used.  Note that there is no consistency in the "Lev"s
      provided for different simulations, nor is there necessarily
      even consistency in the meaning of the "Lev" between different
      simulations (they are not always directly comparable).  Again,
      for reproducibility, it is important to specify the resolution
      of the data you use for a given analysis.

These three pieces of information may be combined into a single string as in
any of the following examples of valid inputs:

    SXS:BBH:0123
    SXS:BBH:0123v2.0
    SXS:BBH:0123/Lev5
    SXS:BBH:0123v2.0/Lev5

The full specification including ID, version, and Lev is called the
"location", but any of these can be provided to load the simulation:

```python
sxs.load("SXS:BBH:0123")
sxs.load("SXS:BBH:0123v2.0")
sxs.load("SXS:BBH:0123/Lev5")
sxs.load("SXS:BBH:0123v2.0/Lev5")
```

## Deprecated or superseded

Many simulations are now quite old, and do not have the benefit of
years of refinements to the simulation code.  As a result, the SXS
collaboration has deprecated many of them, and replaced them with
newer simulations of (nearly) the same physical parameters.

By default, a deprecated simulation raise an error if you attempt to
load it.  However, you can still load it if you want to, by choosing
one of the following options:

1. Pass `ignore_deprecation=True` to completely bypass even checking
   for deprecation or supersession.  No warnings or errors will be
   issued.
2. Include an explicit version number in the `location` string, as in
   "SXS:BBH:0123v2.0".  A warning will be issued that the simulation
   is deprecated, but it will be loaded anyway.
3. Pass `auto_supersede=True` to automatically load the superseding
   simulation, if there is only one.  Because no superseding
   simulation can be *precisely* the same as the deprecated one, there
   may be multiple superseding simulations that have very similar
   parameters, in which case an error will be raised and you must
   explicitly choose one.  If there is only one, a warning will be
   issued, but the superseding simulation will be loaded.
4. Configure `sxs` to automatically load superseding simulations by
   default with `sxs.write_config(auto_supersede=True)`.  This has the
   same effect as passing `auto_supersede=True` to every call to
   `sxs.load`.

Otherwise, a `ValueError` will be raised, with an explanation and
suggestions on what you might want to do.

In this case, "SXS:BBH:0123" is deprecated, and has been superseded by
the much newer simulation "SXS:BBH:2394".  We can load the superseding
simulation as

In [1]:
import sxs

sxs_bbh = sxs.load("SXS:BBH:0123", auto_supersede=True)

We can see that, even though we requested "SXS:BBH:0123", the location of the output
simulation object is "SXS:BBH:2394":

In [2]:
sxs_bbh.location

'SXS:BBH:2394v2.0/Lev3'

Note that the version "v2.0" and "Lev3" were automatically chosen as the highest values, respectively.

## Metadata

At this point, only the metadata (mentioned in the previous notebook) has been loaded, which we can access naturally:

In [3]:
sxs_bbh.metadata

Metadata([('simulation_name', '0123/Lev3'),
          ('alternative_names', ['BFI:Rerun2013Catalog:0123', 'SXS:BBH:2394']),
          ('initial_data_type', 'BBH_SKS'),
          ('object_types', 'BHBH'),
          ('number_of_orbits', 16.2791500016),
          ('reference_mass_ratio', 1.1000208115612666),
          ('reference_chi_eff', -0.1586302306069965),
          ('reference_chi1_perp', 0.2622664068776661),
          ('reference_chi2_perp', 0.06136718678672562),
          ('reference_eccentricity', 0.0002189),
          ('reference_dimensionless_spin1',
           [0.234792297398, 0.133962609266, -0.413950315438]),
          ('reference_dimensionless_spin2',
           [0.0567474431242, -0.030810829516, 0.126303716166]),
          ('reference_orbital_frequency',
           [6.69100038116e-05, -0.000825658162723, 0.0167640334202]),
          ('reference_mass1', 0.523815149031),
          ('reference_mass2', 0.476186580768),
          ('reference_time', 640.0),
          ('reference

Just as `simulations.dataframe` allows us to extract the metadata for all simulations as a uniform `pandas` table, we can extract the metadata for this one simulation in a format consistent with other simulations as a `pandas` series:

In [4]:
import pandas as pd

# The next line is just to ensure the output doesn't look ugly in the docs
with pd.option_context("max_colwidth", 46):
    display(sxs_bbh.series)

object_types                                                                  BHBH
initial_data_type                                                          BBH_SKS
initial_separation                                                            15.0
initial_orbital_frequency                                                 0.015831
initial_adot                                                             -0.000076
initial_ADM_energy                                                        0.992674
initial_ADM_linear_momentum          [-7.9624857e-09, 6.12834433e-08, 4.734964e...
initial_ADM_linear_momentum_mag                                                0.0
initial_ADM_angular_momentum         [0.0796391884578617, -0.0217888728450736, ...
initial_ADM_angular_momentum_mag                                           1.02609
initial_mass1                                                              0.52381
initial_mass2                                                              0.47619
init

Various relevant pieces of information about the simulation are also
available as attributes of the simulation object.  For example, we may
wish to know which versions are available:

In [5]:
sxs_bbh.versions

['', 'v2.0']

In this case, only the generic version, `""`, and a version from the second catalog, `"v2.0"`, are available, because SXS:BBH:2394 is a new simulation.

These versions track modifications to the files representing the data and — together with the SXS ID — establish the unique identifier for the data set.  This unique identifier is also published as a DOI.  The DOI prefix for SXS data is 10.26138, and the full DOI for any simulation is given by combining these:

In [6]:
sxs_bbh.url

'https://doi.org/10.26138/SXS:BBH:2394v2.0'

These DOIs are permanent and can be used to refer to the data in
publications, in the same way that DOIs for journal articles are used.
They point to deposits of the data in the Zenodo repository, which is
a long-term, open-access archive.  This is also where the data are
automatically obtained when you load a particular data set for the
first time.

## Data

Besides the metadata, the remaining data sets are loaded lazily.  This
means that we can access the data as needed, but the cost in time and
resources is not paid unless and until the data is actually accessed.
Specifically, the time to download the data if needed, the disk space
required to cache it if desired, the time to load the data from disk,
and the memory required to store it are all deferred to the point of
use.

For example, we can access data describing the horizons as

In [7]:
sxs_bbh.horizons

And data describing the waveform as

In [8]:
sxs_bbh.h

WaveformModes([[-5.16037020e-02-2.06347735e-02j,  4.20048090e-03-2.37941104e-04j,
        -5.61648391e-03-2.20335263e-04j, ...,
         2.68997649e-05-1.03174161e-04j,  1.06676465e-04-2.17019697e-05j,
        -6.08846226e-05-2.45457325e-05j],
       [-5.39636042e-02-2.17428469e-02j,  4.33165015e-03+1.27649386e-05j,
        -9.07673443e-03-1.93966662e-04j, ...,
        -1.88319693e-06-1.01668419e-04j,  1.06194851e-04+3.20981128e-05j,
         7.02178990e-05-3.28428107e-05j],
       [-5.80508703e-02-2.26188992e-02j,  4.46988000e-03+3.49591765e-04j,
        -1.11737936e-02-1.40045493e-04j, ...,
        -2.82041113e-05-8.77757310e-05j,  8.16792872e-05+7.99667559e-05j,
         1.80576360e-04-2.07837032e-05j],
       ...,
       [ 1.79758572e-04-2.57447253e-04j,  8.88383893e-03-4.28228586e-03j,
         7.20501214e-02-6.52173524e-08j, ...,
         4.24869385e-08-2.13346232e-08j, -2.22454446e-08+4.27985831e-08j,
        -1.40548692e-07+1.21393256e-07j],
       [ 1.79750972e-04-2.57406571e-

The objects returned will be the subject of the next two notebooks in this series:

- [`Horizons`](/tutorials/03-Horizons)
- [`WaveformModes`](/tutorials/04-Waveforms)